# 🏪 店舗別包括ダッシュボード

## 目的
**各店舗の店長が自店舗の全てを理解できる、完全な経営ダッシュボード**

### このダッシュボードで分かること
1. **時系列分析**（月次・週次・日次）
   - 売上トレンド
   - 前期比較
   - 成長率

2. **商品分析**（カテゴリ別ABC分析）
   - 重点商品の特定
   - 死に筋商品の特定
   - カテゴリ別売上構成

3. **発注最適化**
   - 推奨発注量
   - 欠品リスク商品
   - 過剰在庫商品

4. **外部要因分析**
   - 天気と売上の相関
   - イベントの影響
   - 季節性トレンド

5. **店舗間比較**
   - 同規模・同天候での比較
   - 売れ筋商品の違い
   - 在庫管理の違い

6. **店長パフォーマンス**
   - 同条件での売上比較
   - 商品構成の違い
   - 改善ポイント

7. **需要予測**
   - AI予測による今後の売上
   - 推奨アクション

---

In [ ]:
# 日本語フォント設定（共通モジュール）
import warnings
warnings.filterwarnings('ignore')

# よく使うライブラリを先に読み込む
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ウィジェットの有無を通知・フラグ化
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
    print('✅ ipywidgets利用可能')
except Exception:
    WIDGETS_AVAILABLE = False
    print('⚠️ ipywidgets未インストール - 一部機能制限')

import font_setup
JP_FP = font_setup.setup_fonts(show_test=False)


## 📊 1. データ読み込みと店舗選択

In [ ]:
# データ読み込み
file_path = 'output/06_final_enriched_20250701_20250930.csv'
print(f"📂 データ読み込み中: {file_path}")
df = pd.read_csv(file_path, encoding='utf-8-sig', low_memory=False)
df['日付'] = pd.to_datetime(df['日付'])

print(f"✅ データ読み込み完了: {len(df):,}行")
print(f"期間: {df['日付'].min().strftime('%Y-%m-%d')} 〜 {df['日付'].max().strftime('%Y-%m-%d')}")

# 店舗一覧
stores = df['店舗'].unique()
print(f"\n📍 店舗一覧: {len(stores)}店舗")
for i, store in enumerate(stores, 1):
    store_data = df[df['店舗'] == store]
    total_sales = store_data['売上金額'].sum()
    print(f"  {i}. {store} (総売上: ¥{total_sales:,.0f})")

print("\n" + "="*80)
print("⚠️ 【重要】店舗を選択してください")
print("="*80)
print("下のセルで TARGET_STORE を設定してから実行してください")
print("例: TARGET_STORE = '58738:ＴＸ秋葉原駅'")

In [ ]:
# ========================================
# ここで分析対象の店舗を選択
# ========================================
TARGET_STORE = '58738:ＴＸ秋葉原駅'  # ← ここを変更

# 対象店舗のデータを抽出
store_df = df[df['店舗'] == TARGET_STORE].copy()
other_stores_df = df[df['店舗'] != TARGET_STORE].copy()

print("="*80)
print(f"🎯 分析対象店舗: {TARGET_STORE}")
print("="*80)
print(f"対象店舗データ: {len(store_df):,}行")
print(f"他店舗データ: {len(other_stores_df):,}行")
print(f"\n総売上: ¥{store_df['売上金額'].sum():,.0f}")
print(f"総販売数: {store_df['売上数量'].sum():,.0f}個")
print(f"平均客単価: ¥{store_df.groupby('日付')['売上金額'].sum().mean():,.0f}/日")

## 📈 2. 時系列分析（月次・週次・日次）

In [ ]:
print("="*80)
print("📈 時系列分析")
print("="*80)

# 日次売上
daily_sales = store_df.groupby('日付').agg({
    '売上金額': 'sum',
    '売上数量': 'sum'
}).reset_index()

# 週次売上
store_df['週'] = store_df['日付'].dt.to_period('W').astype(str)
weekly_sales = store_df.groupby('週').agg({
    '売上金額': 'sum',
    '売上数量': 'sum'
}).reset_index()

# 月次売上
monthly_sales = store_df.groupby('月').agg({
    '売上金額': 'sum',
    '売上数量': 'sum'
}).reset_index()

print("\n【月次サマリー】")
print(monthly_sales.to_string(index=False))

# 前月比
if len(monthly_sales) > 1:
    monthly_sales['前月比'] = monthly_sales['売上金額'].pct_change() * 100
    print("\n【前月比】")
    for idx, row in monthly_sales.iterrows():
        if pd.notna(row['前月比']):
            print(f"  {int(row['月'])}月: {row['前月比']:+.1f}%")

# グラフ描画
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.25)

# 日次売上推移
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(daily_sales['日付'], daily_sales['売上金額'], linewidth=1, alpha=0.5, color='lightblue')
ax1.plot(daily_sales['日付'], daily_sales['売上金額'].rolling(7, center=True).mean(), 
         linewidth=2.5, color='darkblue', label='7日移動平均')
ax1.set_title(f'📊 {TARGET_STORE} - 日次売上推移', fontsize=16, fontproperties=JP_FP)
ax1.set_xlabel('日付', fontsize=12, fontproperties=JP_FP)
ax1.set_ylabel('売上金額（円）', fontsize=12, fontproperties=JP_FP)
ax1.legend(fontsize=11, prop=JP_FP)
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# 週次売上
ax2 = fig.add_subplot(gs[1, 0])
ax2.bar(range(len(weekly_sales)), weekly_sales['売上金額'], color='steelblue', edgecolor='black')
ax2.set_title('📅 週次売上', fontsize=14, fontproperties=JP_FP)
ax2.set_xlabel('週', fontsize=11, fontproperties=JP_FP)
ax2.set_ylabel('売上金額（円）', fontsize=11, fontproperties=JP_FP)
ax2.set_xticks(range(len(weekly_sales)))
ax2.set_xticklabels([w.split('/')[1] if '/' in w else w for w in weekly_sales['週']], rotation=45)
ax2.grid(axis='y', alpha=0.3)

# 月次売上
ax3 = fig.add_subplot(gs[1, 1])
colors = ['green' if i > 0 else 'red' if i < 0 else 'gray' 
          for i in monthly_sales.get('前月比', [0] * len(monthly_sales))]
ax3.bar(monthly_sales['月'].astype(str), monthly_sales['売上金額'], color=colors, edgecolor='black')
ax3.set_title('📆 月次売上', fontsize=14, fontproperties=JP_FP)
ax3.set_xlabel('月', fontsize=11, fontproperties=JP_FP)
ax3.set_ylabel('売上金額（円）', fontsize=11, fontproperties=JP_FP)
ax3.grid(axis='y', alpha=0.3)

# 曜日別平均売上
weekday_map = {0: '月', 1: '火', 2: '水', 3: '木', 4: '金', 5: '土', 6: '日'}
store_df['曜日名'] = store_df['日付'].dt.dayofweek.map(weekday_map)
weekday_avg = store_df.groupby('曜日名')['売上金額'].mean()
weekday_order = ['月', '火', '水', '木', '金', '土', '日']
weekday_avg = weekday_avg.reindex(weekday_order)

ax4 = fig.add_subplot(gs[2, 0])
colors = ['coral' if day in ['土', '日'] else 'steelblue' for day in weekday_order]
ax4.bar(weekday_order, weekday_avg, color=colors, edgecolor='black')
ax4.set_title('📅 曜日別平均売上', fontsize=14, fontproperties=JP_FP)
ax4.set_xlabel('曜日', fontsize=11, fontproperties=JP_FP)
ax4.set_ylabel('平均売上金額（円）', fontsize=11, fontproperties=JP_FP)
ax4.grid(axis='y', alpha=0.3)

# 時間帯分析（売上数量のヒストグラム）
ax5 = fig.add_subplot(gs[2, 1])
ax5.hist(daily_sales['売上金額'], bins=30, color='teal', edgecolor='black', alpha=0.7)
ax5.axvline(daily_sales['売上金額'].mean(), color='red', linestyle='--', linewidth=2, label='平均')
ax5.axvline(daily_sales['売上金額'].median(), color='orange', linestyle='--', linewidth=2, label='中央値')
ax5.set_title('📊 日次売上分布', fontsize=14, fontproperties=JP_FP)
ax5.set_xlabel('売上金額（円）', fontsize=11, fontproperties=JP_FP)
ax5.set_ylabel('日数', fontsize=11, fontproperties=JP_FP)
ax5.legend(fontsize=11, prop=JP_FP)
ax5.grid(axis='y', alpha=0.3)

plt.show()

print("\n✅ 時系列分析完了")